# Database programming: SQLite

SQLite is an embedded database engine (there is no separate server) that is included with Python.

In [ ]:
import sqlite3
conn = sqlite3.connect('data/real-estate.db')

In [ ]:
cursor = conn.execute('SELECT * FROM "transactions" LIMIT 10')

In [ ]:
cursor.description

In [ ]:
for row in cursor:
    print(row)

In [ ]:
for row in conn.execute('SELECT type, count(*) from transactions GROUP BY type'):
    print(row)

In [ ]:
conn.execute("select * from transactions where type = 'Unkown'").fetchone()

# Please don't do this
```python
user_provided_data = "SACRAMENTO'; DROP TABLE transactions; -- "
format_str = "SELECT * FROM transactions WHERE city = '%s'"" % (user_provided_data)
```

https://xkcd.com/327/

In [ ]:
user_provided_data = "SACRAMENTO'; DROP TABLE transactions; -- "
format_str = "SELECT * FROM transactions WHERE city = '%s'" % (user_provided_data)
format_str

# Do this instead

"Bind" parameters:

In [ ]:
stmt = 'SELECT count(*), avg(price) FROM transactions WHERE type = ?'

In [ ]:
cursor = conn.execute(stmt, ('Condo',))
for row in cursor:
    print(row)

In [ ]:
for row in conn.execute(stmt, ('Condo',)):
    print(row)

In [ ]:
for row in conn.execute(stmt, ('Residential\'); --)',)):
    print(row)

Writing data

In [ ]:
c = conn.cursor()
c.execute("""CREATE TABLE stocks(
    date text, 
    symbol text, 
    price real
)""")

In [ ]:
data = [
    ("2014-01-02", "F", 12.089),
    ("2014-01-02", "TSLA", 150.1),
    ("2014-01-02", "IBM", 157.6001),
    ("2014-01-02", "AAPL", 72.7741),
    ("2014-01-03", "F", 12.1438),
    ("2014-01-03", "TSLA", 149.56),
    ("2014-01-03", "IBM", 158.543),
    ("2014-01-03", "AAPL", 71.1756),
    ("2014-01-06", "F", 12.1986),
    ("2014-01-06", "TSLA", 147.0),
    ("2014-01-06", "IBM", 157.9993),
    ("2014-01-06", "AAPL", 71.5637),
    ("2014-01-07", "F", 12.042),
    ("2014-01-07", "TSLA", 149.36),
    ("2014-01-07", "IBM", 161.1508),
    ("2014-01-07", "AAPL", 71.0516),
    ("2014-01-08", "F", 12.1673),
    ("2014-01-08", "TSLA", 151.28),
    ("2014-01-08", "IBM", 159.6728),
    ("2014-01-08", "AAPL", 71.5019),
]
data = iter(data)

In [ ]:
data

In [ ]:
c.executemany('INSERT INTO stocks VALUES(?, ?, ?)', data)

In [ ]:
conn.commit()

In [ ]:
conn.execute("SELECT * FROM stocks WHERE symbol = 'F'").fetchall()

In [ ]:
conn.execute("UPDATE stocks SET price=0 WHERE symbol='TSLA'")

In [ ]:
conn.execute("SELECT * FROM stocks WHERE symbol = 'TSLA'").fetchall()

In [ ]:
with conn:
    conn.execute("DELETE FROM stocks WHERE symbol = 'TSLA'")

In [ ]:
c.execute("DROP TABLE stocks")

# Database programming: MySQL

To use MySQL, we'll need to install the pymysql driver:

In [ ]:
!pip install pymysql

In [ ]:
!pip freeze | grep -i pymysql

In [ ]:
import pymysql

host = 'database-1.c9zriyyu6mtc.us-east-1.rds.amazonaws.com'
conn = pymysql.connect(
    host=host,
    user='newuser',
    password='password',
    db='class',
)

In [ ]:
with conn.cursor() as c:
    print(c, type(c))
    value = c.execute('SELECT type, count(*) from transactions GROUP BY type')
    print(c.description)
    print(value)
    for item in c:
        print(item)

In [ ]:
conn

In [ ]:
with conn.cursor() as c:
    c.execute("""CREATE TABLE IF NOT EXISTS stocks(
        date text, 
        symbol text, 
        price real
    )""")

In [ ]:
with sqlite3.connect('./data/stocks.db') as sqlite_conn:
    data = list(sqlite_conn.execute('SELECT date, symbol, price FROM stocks_data'))

In [ ]:
data[:1]

In [ ]:
sqlite_conn = sqlite3.connect('./data/stocks.db')

with conn.cursor() as mysql_cursor:
    sqlite_cursor = sqlite_conn.cursor()
    data = sqlite_cursor.execute('SELECT date, symbol, price FROM stocks_data')
    mysql_cursor.execute('DELETE FROM stocks;')
    mysql_cursor.executemany('INSERT INTO stocks VALUES(%s, %s, %s)', data)

In [ ]:
with conn.cursor() as c:
    q = c.execute("SELECT symbol, count(*) FROM stocks group by symbol")
    print(list(c))


In [ ]:
with conn.cursor() as c:
    q = c.execute("SELECT * FROM stocks WHERE symbol = 'IBM' LIMIT 5")
    print(c.description)
    for row in c:
        print(row)


In [ ]:
with conn.cursor() as c:
    c.execute("DELETE FROM stocks WHERE symbol = 'TSLA'")

In [ ]:
with conn.cursor() as c:
    c.execute("DROP TABLE stocks")

Open [DBAPI lab](./dbapi-lab.ipynb)